## Import dependent libraries

In [ ]:
from pandas import read_csv
from matplotlib import pyplot
import math
from numpy import concatenate, array, asarray
from pandas import DataFrame
from pandas import concat
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Model, model_from_json
from keras.layers import Input
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.layers.core import Activation
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LambdaCallback
from keras.utils import plot_model
from sklearn.externals import joblib
import tensorflow as tf
from keras.utils import plot_model
import keras.backend as K
import random


## Parameters

In [2]:
# specify the number of lag hours
n_hours = 128
n_features = 2
n_train_hours = 200
n_divisions = 4
batch_size = n_hours
epochs = 100

## Load dataset

In [3]:
params = {'num': 0}
setattr(K, 'params', params)

In [4]:
branch_input_size = int(n_hours)
branch_output_size = int(n_hours/n_divisions/n_divisions)
print(branch_input_size)
print(branch_output_size)

128
8


In [5]:
def splitter(x):
    num = K.params['num']
    steps = int(n_hours/n_divisions)
    start = int(num * steps)
    end = int((num+1) * steps)
    x = x[:,start:end]
    print(start,end)
    return x

def split_shape(input_shape):
    return (input_shape[0], int(input_shape[1]/n_divisions), input_shape[2])

def timeseries_model(num_steps, num_features):
    visible = Input(name='input', shape=(num_steps, num_features))
    print(visible)
    for i in range(0, n_divisions):
        params = {'num': i}
        setattr(K, 'params', params)
        vars()["lambda"+str(i)] = keras.layers.Lambda(splitter,
                         output_shape=split_shape)(visible)
        vars()["hidden"+str(i)] = LSTM(branch_input_size, return_sequences=True)(vars()["lambda"+str(i)])

        vars()["batchNorm"+str(i)] = BatchNormalization()(vars()["hidden"+str(i)])

        vars()["dropout"+str(i)] = Dropout(0.5)(vars()["batchNorm"+str(i)])

        vars()["output"+str(i)] = LSTM(branch_output_size, return_sequences=True)(vars()["dropout"+str(i)])
        print(vars()["output"+str(i)])
        
    concat = keras.layers.concatenate([output0, output1, output2, output3])
    print(concat)
    tdd = keras.layers.TimeDistributed(Dense(branch_input_size))(concat)
    hidden_Z = LSTM(branch_input_size)(tdd)

    output_notes = Dense(1, activation='sigmoid', name='output_notes')(hidden_Z)
    output_volume = Dense(1, activation='sigmoid', name='output_volume')(hidden_Z)

    model = Model(inputs=[visible], outputs=[
                                             output_notes, 
                                             output_volume, 
                                            ])

    optimizer = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='mae', optimizer=optimizer)
        
    return model

In [ ]:

# model.summary()

In [ ]:
import pydot
import graphviz
plot_model(model, to_file='model.png')


In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
test_data = [[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[ 0,  0],
[84,  1],
[84,  0],
[86, 20],
[89, 48],
[89,  0],
[90, 41],
[91, 50],
[91,  0],
[92, 65],
[92, 72],
[92, 72],
[92, 69],
[80, 69],
[80, 66],
[80, 66],
[92, 60],
[92, 64],
[92, 52],
[92, 52],
[92, 56],
[92, 59],
[92, 59],
[92, 59],
[92, 60],
[92, 60],
[93, 61],
[93, 57],
[93, 57],
[93, 57],
[92, 59],
[92,  0],
[92, 70],
[92, 71],
[92, 72],
[92, 72],
[92, 72],
[92, 73],
[92, 73],
[92, 73],
[92, 73],
[92, 73],
[92, 74],
[92, 74],
[92, 74],
[92, 74],
[92, 73],
[93, 73],
[93,  0],
[93, 58],
[92, 60],
[92,  0],
[92, 67],
[92, 70],
[92, 70],
[92, 69],
[92, 71],
[92, 71],
[92, 73],
[91, 75],
[91,  0],
[90, 91],
[89, 90],
[89, 87],
[89, 87],
[90, 91],
[90, 91],
[90, 91],
[90, 88],
[90, 89],
[90, 89],
[90, 91],
[90, 91],
[90, 91],
[90, 91],
[90, 90],
[90, 89],
[90, 89],
[90, 89],
[90, 89],
[78, 89],
[78,  0],
[90, 82],
[90, 94],
[90, 94],
[78, 93],
[90, 90],
[90,  0],
[78, 82],
[90, 85],
[90, 85],
[90, 85],
[90, 82],
[90, 81],
[90, 81],
[90, 81],
[90, 82],
[90, 82],
[90, 82],
[90, 82],
[90, 80],
[90, 80],
[90, 79],
[90, 79],
[90, 80],
[90, 80],
[90, 80],
[90, 81],
[90, 81],
[90, 81],
[90, 81],
[90, 81],
[90, 81],
[90, 81]]

In [ ]:
testx = DataFrame(data = test_data)
testx = testx.values
testx = testx.astype('float32')
testx = scaler.transform(testx)
testx = series_to_supervised(testx, n_hours, 1)
print(testx)
testx = testx.values
testx = testx[:, :n_obs]
testx = testx.reshape((testx.shape[0], n_hours, n_features))
print(testx)

In [ ]:
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating sound after: %d' % epoch)

    
    print('input_values:', input_values[20:30])
    prediction = model.predict(test)
    print(prediction)
#     prediction = asarray(prediction).ravel().reshape(-1,2)
#     prediction = scaler.inverse_transform(prediction)
#     prediction = prediction.astype('int')
# #     print("INPUT_VALUES:")
# #     print(input_values)
#     print("PREDICTION:")
#     print(prediction[0])
#     print("Note:", prediction[0][0], "Volume:", prediction[0][1])
    
play_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
history = model.fit(
            {'input': train_X},
            {
                'output_notes': train_y_notes, 
                'output_volume': train_y_volume, 
#                 'output_length': train_y_length
            },
            validation_data=({'input': test_X},
                             {
                                 'output_notes': test_y_notes, 
                                 'output_volume': test_y_volume, 
#                                  'output_length': test_y_length
                             }), 
            verbose=1,
            shuffle=False,
            epochs=epochs, 
            batch_size=batch_size,
            callbacks=[play_callback]
)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

###### 